# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [51]:
import os
import glob
import psycopg2
import pandas as pd
import numpy as np
from sql_queries import *
from datetime import datetime

In [54]:
#cur.execute("DROP DATABASE IF EXISTS sparkifydb")
#cur.execute("CREATE DATABASE sparkifydb WITH ENCODING 'utf8' TEMPLATE template0")
#conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
#cur = conn.cursor()

 # connect to default database
conn = psycopg2.connect("host=127.0.0.1 dbname=studentdb user=student password=student")
conn.set_session(autocommit=True)
cur = conn.cursor()
    
    # create sparkify database with UTF8 encoding
cur.execute("DROP DATABASE IF EXISTS sparkifydb1")
cur.execute("CREATE DATABASE sparkifydb1 WITH ENCODING 'utf8' TEMPLATE template0")

    # close connection to default database
conn.close()    
    
    # connect to sparkify database
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb1 user=student password=student")
cur = conn.cursor()
    

In [55]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [56]:
song_files = get_files('data/song_data/A')


In [57]:
filepath = song_files[0]

In [58]:
#Read JSON file through PANDAS function read_json
data = pd.read_json(filepath,lines=True)
print(data)

            artist_id  artist_latitude  artist_location  artist_longitude  \
0  ARD7TVE1187B99BFB1              NaN  California - LA               NaN   

  artist_name   duration  num_songs             song_id             title  \
0      Casual  218.93179          1  SOMZWCG12A8C13C480  I Didn't Mean To   

   year  
0     0  


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [59]:
#Convert data to a Data Frame for easy execution & print values
df = pd.DataFrame(data)
print(df)

            artist_id  artist_latitude  artist_location  artist_longitude  \
0  ARD7TVE1187B99BFB1              NaN  California - LA               NaN   

  artist_name   duration  num_songs             song_id             title  \
0      Casual  218.93179          1  SOMZWCG12A8C13C480  I Didn't Mean To   

   year  
0     0  


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [60]:
#For test purposes only, CREATE table
try:
    cur.execute("CREATE TABLE IF NOT EXISTS songs3(songplay_id varchar, title varchar, artist_id varchar, year int, duration float)")
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)
    
#Insert dataframe data using function ITERTUPLES() in a for loop
try:
     for row in df.itertuples():
         cur.execute("INSERT INTO songs3(songplay_id, title, artist_id, year, duration)\
                      VALUES (%s,%s,%s,%s,%s)",\
                      (row.song_id,
                       row.title,
                       row.artist_id,
                       row.year,
                       row.duration)
                   ) 

except psycopg2.Error as e: 
    print("Error: INsert error")
    print (e)


conn.commit()

#Check if values have been correctly inserted & print them out
cur.execute("SELECT * FROM songs3;")
row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

print(df.head())

('SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179)
            artist_id  artist_latitude  artist_location  artist_longitude  \
0  ARD7TVE1187B99BFB1              NaN  California - LA               NaN   

  artist_name   duration  num_songs             song_id             title  \
0      Casual  218.93179          1  SOMZWCG12A8C13C480  I Didn't Mean To   

   year  
0     0  


Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [61]:
#For test purposes only, CREATE table
try:
    cur.execute("CREATE TABLE IF NOT EXISTS artists(artist_id varchar,name text, location varchar,latitude float, longitude float)")
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)
    
#Insert dataframe data using function ITERTUPLES() in a for loop
try:
     for row in df.itertuples():
         cur.execute("INSERT INTO artists(artist_id,name,location,latitude,longitude)\
                      VALUES (%s,%s,%s,%s,%s)",\
                      (row.artist_id,
                       row.artist_name,
                       row.artist_location,
                       row.artist_latitude,
                       row.artist_longitude)
                   ) 

except psycopg2.Error as e: 
    print("Error: INsert error")
    print (e)


conn.commit()

#Check if values have been correctly inserted & print them out
cur.execute("SELECT * FROM artists;")
row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()


('ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan)


#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [62]:
#Get all the files from the log folder
log_files = get_files('data/log_data/2018/11')


In [63]:
#Pick up the first file in the log folder
filepath = log_files[0]

In [64]:
#Using PANDA functions, read through the contents of the first file and display that
df_log = pd.read_json(filepath, lines=True)
print(df_log)

                                      artist        auth firstName gender  \
0                              Stephen Lynch   Logged In    Jayden      M   
1                                    Manowar   Logged In     Jacob      M   
2                                  Morcheeba   Logged In     Jacob      M   
3                                   Maroon 5   Logged In     Jacob      M   
4                                      Train   Logged In     Jacob      M   
5                                      LMFAO   Logged In     Jacob      M   
6                                   DJ Dizzy   Logged In     Jacob      M   
7                    Fish Go Deep & Tracey K   Logged In     Jacob      M   
8                                       None   Logged In    Alivia      F   
9                                        M83   Logged In     Jacob      M   
10                                    Saybia   Logged In     Jacob      M   
11                             Local Natives   Logged In     Jacob      M   

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [65]:
#Filter records by NextSong action & store it in a variable df_filter
df_filter = df_log[(df_log['page'] == 'NextSong')]


In [66]:
#Convert data in ts column to date and time. Create another column timestamp and add that to the dataframe.    
df_filter['timestamp'] = (pd.to_datetime(df_filter['ts'], unit='ms'))
print(df_filter['timestamp'])

0     2018-11-30 00:22:07.796
1     2018-11-30 01:08:41.796
2     2018-11-30 01:12:48.796
3     2018-11-30 01:17:05.796
4     2018-11-30 01:20:56.796
5     2018-11-30 01:24:32.796
6     2018-11-30 01:28:19.796
7     2018-11-30 01:32:00.796
9     2018-11-30 01:38:17.796
10    2018-11-30 01:39:53.796
11    2018-11-30 01:44:10.796
12    2018-11-30 01:48:36.796
13    2018-11-30 01:50:28.796
14    2018-11-30 01:55:35.796
15    2018-11-30 02:01:08.796
16    2018-11-30 02:05:24.796
17    2018-11-30 02:10:21.796
18    2018-11-30 02:15:54.796
19    2018-11-30 02:19:19.796
23    2018-11-30 03:06:30.796
24    2018-11-30 03:10:30.796
25    2018-11-30 03:13:03.796
26    2018-11-30 03:15:41.796
27    2018-11-30 03:21:09.796
30    2018-11-30 03:29:23.796
31    2018-11-30 03:32:46.796
33    2018-11-30 03:37:24.796
35    2018-11-30 03:39:31.796
36    2018-11-30 03:44:28.796
38    2018-11-30 03:49:50.796
                ...          
356   2018-11-30 17:49:45.796
357   2018-11-30 17:53:44.796
358   2018

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [67]:
#timedata = []

#Convert timestamp column into multiple data frame columns
#df_filter['YY']=df_filter.timestamp.dt.year()
#print(df_filter['YY'])

#Add Timestamp to a list
#timedata = ((df_filter.ts.head()).tolist())

#Add Year to Timedata list
#timedata.append((df_filter.timestamp.dt.year.head()).tolist())

#Add Month to Timedata list
#timedata.append((df_filter.timestamp.dt.month.head()).tolist())


#Add Day to Timedata list
#timedata.append((df_filter.timestamp.dt.day.head()).tolist())


#Add Hour to Timedata list
#timedata.append((df_filter.timestamp.dt.hour.head()).tolist())


#Add Week Of Year to Timedata list
#timedata.append((df_filter.timestamp.dt.week.head()).tolist())

#Add Week Day to Timedata list
#timedata.append((df_filter.timestamp.dt.weekday_name.head()).tolist())
#print(timedata)

#column_labels = []
#column_lables = ['Timestamp','Year']
#print(column_labels)

#Extract individual row content from Data Frame and then append it a list

time_data = [ df_filter["timestamp"], df_filter["timestamp"].dt.hour, df_filter["timestamp"].dt.day, df_filter["timestamp"].dt.weekofyear, df_filter["timestamp"].dt.month, df_filter["timestamp"].dt.year, df_filter["timestamp"].dt.weekday,df_filter["timestamp"].dt.time]
#print(time_data)
column_labels = ['ts','hr','Day','WoY','mm','yy','weekday','start_time']
time_data_dict=dict(zip(column_labels,time_data))
#print(time_data_dict)

time_df=pd.DataFrame.from_dict(time_data_dict)
print(time_df.head())
#print(type(time_df['start_time']))
#except psycopg2.Error as e: 
#    print("Error: INsert error")
#    print (e)

                       ts  hr  Day  WoY  mm    yy  weekday       start_time
0 2018-11-30 00:22:07.796   0   30   48  11  2018        4  00:22:07.796000
1 2018-11-30 01:08:41.796   1   30   48  11  2018        4  01:08:41.796000
2 2018-11-30 01:12:48.796   1   30   48  11  2018        4  01:12:48.796000
3 2018-11-30 01:17:05.796   1   30   48  11  2018        4  01:17:05.796000
4 2018-11-30 01:20:56.796   1   30   48  11  2018        4  01:20:56.796000


In [ ]:
#time_df = pd.DataFrame(timedata,columns=column_labels)
#print(time_df)
#time_df.head()

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [43]:
#For test purposes only, CREATE table
try:
    cur.execute("CREATE TABLE IF NOT EXISTS time(start_time time, hour int, day int,week int, month int, year int, weekday int)")
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)
    
#Insert dataframe data using function ITERTUPLES() in a for loop
try:
     for row in time_df.itertuples():
         cur.execute("INSERT INTO time(start_time,hour,day,week,month,year,weekday)\
                      VALUES (%s,%s,%s,%s,%s,%s,%s)",\
                      (row.start_time,
                       row.hr,
                       row.Day,
                       row.WoY,
                       row.mm,
                       row.yy,
                       row.weekday)
                   ) 

except psycopg2.Error as e: 
    print("Error: INsert error")
    print (e)


conn.commit()

#Check if values have been correctly inserted & print them out
#cur.execute("SELECT * FROM time;")
#row = cur.fetchone()
#while row:
#   print(row)
#   row = cur.fetchone()


Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [45]:
#df_log contains user_id, f_name,l_name,gender,level. We need to transfer those values to the user data frame
user_df1 = df_log[['userId','firstName','lastName','gender','level']].copy()
user_df=user_df1.dropna()
user_df=user_df.drop_duplicates()
print(user_df)

    userId  firstName   lastName gender level
0       91     Jayden       Bell      M  free
1       73      Jacob      Klein      M  paid
8        4     Alivia    Terrell      F  free
22      86      Aiden       Hess      M  free
28      24      Layla    Griffin      F  paid
40      26       Ryan      Smith      M  free
59      49      Chloe     Cuevas      F  paid
62      57  Katherine        Gay      F  free
67      30      Avery    Watkins      F  paid
78      92      Ryann      Smith      F  free
95      69   Anabelle    Simpson      F  free
125     74     Braden     Parker      M  free
134     88   Mohammad  Rodriguez      M  paid
145     36    Matthew      Jones      M  paid
168      8     Kaylee    Summers      F  free
177     12     Austin    Rosales      M  free
184     61     Samuel   Gonzalez      M  free
194     43     Jahiem      Miles      M  free
196     68     Jordan  Rodriguez      F  free
204     85    Kinsley      Young      F  paid
213     50        Ava   Robinson  

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [46]:
#For test purposes only, CREATE table
try:
    cur.execute("CREATE TABLE IF NOT EXISTS users2(user_id int, fname text, lname text,gender text,level varchar)")
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)
    
'''for row in user_df.itertuples():
    cur.execute("INSERT INTO users(user_id,fname,lname,gender,level)\
                 VALUES (%s,%s,%s,%s,%s)",\
                 (row.userId,
                  row.firstName,
                  row.lastName,
                  row.gender,
                  row.level)
               )
          
conn.commit()
'''    
try:
     for row in user_df.itertuples():
         cur.execute("INSERT INTO users2(user_id,fname,lname,gender,level)\
                      VALUES (%s,%s,%s,%s,%s)",\
                      (row.userId,
                      row.firstName,
                      row.lastName,
                      row.gender,
                      row.level)
                    ) 

except psycopg2.Error as e: 
    print("Error: INsert error")
    print (e)


conn.commit()
#Check if values have been correctly inserted & print them out

cur.execute("SELECT * FROM users2;")
row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()



(91, 'Jayden', 'Bell', 'M', 'free')
(73, 'Jacob', 'Klein', 'M', 'paid')
(4, 'Alivia', 'Terrell', 'F', 'free')
(86, 'Aiden', 'Hess', 'M', 'free')
(24, 'Layla', 'Griffin', 'F', 'paid')
(26, 'Ryan', 'Smith', 'M', 'free')
(49, 'Chloe', 'Cuevas', 'F', 'paid')
(57, 'Katherine', 'Gay', 'F', 'free')
(30, 'Avery', 'Watkins', 'F', 'paid')
(92, 'Ryann', 'Smith', 'F', 'free')
(69, 'Anabelle', 'Simpson', 'F', 'free')
(74, 'Braden', 'Parker', 'M', 'free')
(88, 'Mohammad', 'Rodriguez', 'M', 'paid')
(36, 'Matthew', 'Jones', 'M', 'paid')
(8, 'Kaylee', 'Summers', 'F', 'free')
(12, 'Austin', 'Rosales', 'M', 'free')
(61, 'Samuel', 'Gonzalez', 'M', 'free')
(43, 'Jahiem', 'Miles', 'M', 'free')
(68, 'Jordan', 'Rodriguez', 'F', 'free')
(85, 'Kinsley', 'Young', 'F', 'paid')
(50, 'Ava', 'Robinson', 'F', 'free')
(6, 'Cecilia', 'Owens', 'F', 'free')
(101, 'Jayden', 'Fox', 'M', 'free')
(16, 'Rylan', 'George', 'M', 'paid')
(78, 'Chloe', 'Roth', 'F', 'free')
(33, 'Bronson', 'Harris', 'M', 'free')
(5, 'Elijah', 'Davi

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [50]:
for index, row in df.iterrows():
#get songid and artistid from song and artist tables
    cur.execute(song_select,(row.title, row.artist_name, row.duration))
#    results = cur.fetchone()
    print(cur.fetchone())   
#    if results:
#        songid, artistid = results
#    else:
#        songid, artistid = None, None

# insert songplay record
#    songplay_data = ()
#    cur.execute(songplay_table_insert, songplay_data)
#conn.commit()

InternalError: current transaction is aborted, commands ignored until end of transaction block


Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.